In [1]:
# Code from the channel 'computer science' on Youtube

from bs4 import BeautifulSoup
import pandas as pd
import requests
import urllib.request
import time

In [2]:
authors = []
dates = []
statements = []
sources = []
targets = []

In [3]:
def scrape_website(page_number):
  page_num = str(page_number)
  URL = 'https://www.politifact.com/factchecks/list/?page='+page_num
  webpage = requests.get(URL)
  soup = BeautifulSoup(webpage.text, 'html.parser')
  statement_footer = soup.find_all ('footer' , attrs={'class' : 'm-statement__footer'})
  statement_quote = soup.find_all ('div' , attrs={'class' : 'm-statement__quote'})
  statement_meta = soup.find_all ('div' , attrs={'class' : 'm-statement__meta'})
  target = soup.find_all ('div' , attrs={'class' : 'm-statement__meter'})

  for i in statement_footer:
    link1 = i.text.strip()
    name_and_date= link1.split()
    first_name = name_and_date[1]
    last_name = name_and_date[2]
    full_name = first_name + ' ' + last_name
    month = name_and_date[4]
    day = name_and_date[5]
    year = name_and_date[6]
    date = month+' '+day+' '+year
    dates.append(date)
    authors.append(full_name)   

  for i in statement_quote:
    link2 = i.find_all('a')
    statement_text = link2[0].text.strip()
    statements.append(statement_text)

  for i in statement_meta:
    link3 = i.find_all('a')
    source_text = link3[0].text.strip()
    sources.append(source_text)

  for i in target:
    link4 = i.find('div' , attrs={'class' : 'c-image'}).find('img').get('alt')
    targets.append(link4)

In [13]:
n = 50
for i in range(1, n):
  scrape_website(i)

In [14]:
data = pd.DataFrame(columns = ['author','statement','source','date','target'])
data['author']= authors
data['statement']= statements
data['source']= sources
data['date']= dates
data['target']= targets

In [15]:
data.head()

,author,statement,source,date,target
0,Andy Nguyen,"""Ivermectin sterilizes the majority (85%) of t...",Facebook posts,"September 9, 2021",false
1,Gabrielle Settles,Says Nancy Pelosi on Aug. 31 blocked members o...,Facebook posts,"September 9, 2021",barely-true
2,Jon Greenberg,“The frequency of hurricanes making landfall i...,Bjorn Lomborg,"September 9, 2021",half-true
3,Tom Kertscher,“President Putin bans mandatory vaccinations i...,Instagram posts,"September 9, 2021",false
4,Amy Sherman,The Texas abortion law “provides at least six ...,Greg Abbott,"September 9, 2021",false


In [16]:
data.shape

(1620, 5)

In [25]:
true = ['mostly-true', 'true']

def  getBinaryNumTarget(text):
     if text in true  :
       return 0
     else:
       return 1

In [26]:
data['BinaryNumTarget']= data['target'].apply(getBinaryNumTarget)


In [27]:
data.head(20)

,author,statement,source,date,target,BinaryNumTarget
0,Andy Nguyen,"""Ivermectin sterilizes the majority (85%) of t...",Facebook posts,"September 9, 2021",false,1
1,Gabrielle Settles,Says Nancy Pelosi on Aug. 31 blocked members o...,Facebook posts,"September 9, 2021",barely-true,1
2,Jon Greenberg,“The frequency of hurricanes making landfall i...,Bjorn Lomborg,"September 9, 2021",half-true,1
3,Tom Kertscher,“President Putin bans mandatory vaccinations i...,Instagram posts,"September 9, 2021",false,1
4,Amy Sherman,The Texas abortion law “provides at least six ...,Greg Abbott,"September 9, 2021",false,1
5,Ciara O'Rourke,"“Germany halts all Covid-19 vaccines, says the...",Bloggers,"September 9, 2021",false,1
6,Ciara O'Rourke,Photo shows a Chinese restaurant sign that say...,Instagram posts,"September 9, 2021",pants-fire,1
7,Ciara O'Rourke,“Military arrests Hunter Biden!”,Bloggers,"September 8, 2021",pants-fire,1
8,Tom Kertscher,Says a demonstration with an air quality monit...,Facebook posts,"September 8, 2021",false,1
9,Louis Jacobson,"Texas “ranks 50th in baby wellness checks, ran...",Occupy Democrats,"September 7, 2021",half-true,1


In [29]:
data.to_csv('fact_checker.csv')